<a href="https://colab.research.google.com/github/rafaelnduarte/Airbnb_Data_Analysis_NYC-XGBoost_Price_Prediction/blob/master/Predict_Future_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predict Future Sales

[![LinkedIn](https://img.shields.io/badge/LinkedIn-rafaelnduarte-blue.svg)](https://www.linkedin.com/in/rafael-n-duarte)
[![GitHub](https://img.shields.io/badge/GitHub-rafaelnduarte-purple.svg)](https://github.com/rafaelnduarte)


This is the final project for the [Coursera]('https://www.coursera.org/') course [How To Win A Data Science Competition]('https://www.coursera.org/learn/competitive-data-science'). It's an advanced Data Science course, created by top [Kaggle]('https://kaggle.com/') Competitors on the tools and techniques they've used, and insights they've learned competing on Kaggle and other platforms.


<center><img width="60%" src="https://image.freepik.com/free-vector/consumers-isometric-composition_1284-26384.jpg"></center>

At the end of the course, [this competition]('https://www.kaggle.com/c/competitive-data-science-predict-future-sales') is assigned as the final project. Here, we will work with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Russian software firms - [1C Company]('https://1c.ru/eng/title.htm'). 

> The goal is to predict total sales for every product and store in the next month.

With that in mind, let's get started!



In [0]:
# importing the necessary packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import scipy.stats as scs

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [0]:
# loading the data into dataframes
i_cat = pd.read_csv('https://raw.githubusercontent.com/rafaelnduarte/Predict_Future_Sales/master/data/item_categories-translated.csv')
items = pd.read_csv('https://raw.githubusercontent.com/rafaelnduarte/Predict_Future_Sales/master/data/items-translated.csv')
sample_sub = pd.read_csv('https://raw.githubusercontent.com/rafaelnduarte/Predict_Future_Sales/master/data/sample_submission.csv')
shops = pd.read_csv('https://raw.githubusercontent.com/rafaelnduarte/Predict_Future_Sales/master/data/shops-translated.csv')
test = pd.read_csv('https://raw.githubusercontent.com/rafaelnduarte/Predict_Future_Sales/master/data/test.csv')
t1 = pd.read_csv('https://raw.githubusercontent.com/rafaelnduarte/Predict_Future_Sales/master/data/sales_train-1.csv')
t2 = pd.read_csv('https://raw.githubusercontent.com/rafaelnduarte/Predict_Future_Sales/master/data/sales_train-2.csv')
t3 = pd.read_csv('https://raw.githubusercontent.com/rafaelnduarte/Predict_Future_Sales/master/data/sales_train-3.csv')
t4 = pd.read_csv('https://raw.githubusercontent.com/rafaelnduarte/Predict_Future_Sales/master/data/sales_train-4.csv')
t5 = pd.read_csv('https://raw.githubusercontent.com/rafaelnduarte/Predict_Future_Sales/master/data/sales_train-5.csv')
t6 = pd.read_csv('https://raw.githubusercontent.com/rafaelnduarte/Predict_Future_Sales/master/data/sales_train-6.csv')

# putting the train dataset together
train = pd.DataFrame(pd.concat([t1,t2,t3,t4,t5,t6]))

## The Data

They provided us with a great amount of data.

We have 6 different datasets:
> * Item Categories
* Items
* Shops
* Test
* Train
* Sample Submission

The **Train** dataset was too big for GitHub, which only accepts file smaller than 25MB.

My intention is to always make all of my code and data available, so to have it on GitHub, I decided to split it into smaller datasets, that were concatenated now, regaining their original shape.

In [34]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [39]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [35]:
sample_sub.tail()

,ID,item_cnt_month
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5
214199,214199,0.5


In [36]:
shops.head()

,shop_id,shop_name_translated
0,0,"Yakutsk Ordzhonikidze, 56 francs"
1,1,"Yakutsk TC ""Central"" franc"
2,2,"Adygea Shopping Center ""Mega"""
3,3,"Balashikha TRK ""October-Kinomir"""
4,4,"Volzhsky shopping center ""Volga Mall"""


In [37]:
items.head()

,item_id,item_name_translated
0,0,!! IN THE POWER OF HAPPINESS (PLAST) D
1,1,! ABBYY FineReader 12 Professional Edition Ful...
2,2,*** IN THE GLORY OF THE GLORY (UNV) D
3,3,*** BLUE WAVE (Univ) D
4,4,*** BOX (GLASS) D


In [38]:
i_cat.head()

,item_category_id,item_category_name_translated
0,0,PC - Headsets / Headphones
1,1,Accessories - PS2
2,2,Accessories - PS3
3,3,Accessories - PS4
4,4,Accessories - PSP


In [0]:
#@title Translating from Russian
# translating from Russian
#!pip install googletrans -q

#from googletrans import Translator
#translator = Translator()
#items['shop_name'] = shops['shop_name'].apply(translator.translate, src='ru', dest='en').apply(getattr, args=('text',))

# saving files

#from google.colab import files
#shops.to_csv("shops_english.csv", index=False)
#files.download("shops_english.csv")
#shops.head()

